In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import gzip
import os

In [3]:
import mediator_was.processing.gtex as gtex
import mediator_was.processing.predixcan as predixcan
import mediator_was.prediction as prediction
import yaml

In [ ]:
analysis_dir = "../../analysis/2015-12-13"

In [4]:
config = yaml.load(open('../../../config.yaml'))

In [5]:
config

{'data': {'gtex': {'dir': '/projects/gtex/data',
   'files': {'expression': 'ExpressionFiles/gtex.portal.normalized/Whole_Blood_Analysis.expr.txt',
    'genotypes': 'GenotypeFiles/450-Individuals/2015-01-12/GTEx_Analysis_20150112_OMNI_2.5M_5M_450Indiv_chr1to22_phased_genot_imput_info04_maf01_HWEp1E6_ConstrVarIDs.vcf.gz'}},
  'predixcan': {'dir': '/home/kbhutani/projects/mediator/data/predixcan/',
   'files': {'database': 'DGN-WB_0.5.txt'}},
  'references': {'dbsnp': '/projects/references/snp142Common.txt.gz'}}}

## Model: Alpha-1

#### Process and load Gustavo Data

In [6]:
# Add positions to database
new_db = os.path.join(predixcan.main_dir, "EN.withpos.tsv")
#predixcan.add_positions_to_database(fn=new_db)

In [7]:
!ls -lh $new_db

-rw-r--r--. 1 kbhutani schork 38M Dec 10 13:01 /home/kbhutani/projects/mediator/data/predixcan/EN.withpos.tsv


In [8]:
# Load database
en_df = predixcan.load_database(new_db, alpha=1)

In [9]:
en_df.head()

,gene,rsid,refAllele,alt,beta,alpha,id,chr,start,end,chromosome,position
0,A2LD1,rs11069419,T,C,-0.034054,1,1359,chr13,101561096,101561097,chr13,101561097
1,A2LD1,rs11842969,C,T,-0.381005,1,1357,chr13,101214374,101214375,chr13,101214375
2,A2LD1,rs1283142,C,A,-0.013816,1,1358,chr13,101392263,101392264,chr13,101392264
3,A2LD1,rs1283211,C,T,0.022643,1,1358,chr13,101369270,101369271,chr13,101369271
4,TMTC4,rs1283211,C,T,0.006821,1,1358,chr13,101369270,101369271,chr13,101369271


### Predicted Values

#### TODO: Unit Test for Predicted Values

In [ ]:
genotype_file = gtex.genotype_fn
predictions_df = prediction.helpers.stream_predict(en_df, genotype_file)

#### Note that it did not compute on the HLA related genes. Let's remove them from further analysis.

In [ ]:
hla_related_genes = set(en_df[en_df['chromosome'] == 25]['gene'])

In [ ]:
full_predicted_genes = set(predictions_df.index) - hla_related_genes
full_predicted_df = predictions_df.ix[full_predicted_genes]
full_predicted_df.columns = full_predicted_df.columns.map(lambda x: "-".join(x.split('-')[:2]))

In [ ]:
full_predicted_df.head()

## Model Alpha=0.5

In [29]:
# Load database
en_df = predixcan.load_database(new_db, alpha=0.5)
genotype_file = gtex.genotype_fn
predictions_df = prediction.helpers.stream_predict(en_df, genotype_file)

Total 331599
Not Found 13
Incorrect 0
Switched 0


In [30]:
pd.concat([pd.DataFrame(x).T for x in not_found])

,gene,rsid,refAllele,alt,beta,alpha,id,chr,start,end,chromosome,position
15588,CDC7,rs10801817,C,T,-0.002368292,0.5,1279,chr1,91001088,91001089,1,91001089
32141,HHIPL2,rs34651530,A,G,-0.05389286,0.5,2286,chr1,222985196,222985197,1,222985197
66724,C3orf52,rs10934162,C,T,0.005354781,0.5,1439,chr3,112001324,112001325,3,112001325
124752,CDK13,rs2329611,C,T,0.003770901,0.5,890,chr7,40008948,40008949,7,40008949
127761,ZP3,rs17149187,A,G,0.03886865,0.5,1164,chr7,75998572,75998573,7,75998573
193217,MED17,rs11020719,G,A,-0.01918575,0.5,1302,chr11,94008519,94008520,11,94008520
209529,KRR1,rs1026784,G,A,-0.03149808,0.5,1164,chr12,75997732,75997733,12,75997733
221980,CDC16,rs7994556,C,T,0.03752229,0.5,1462,chr13,114998075,114998076,13,114998076
229195,FCF1,rs17183663,C,A,0.001574773,0.5,1164,chr14,76001456,76001457,14,76001457
256578,ZNF469,rs4843756,C,T,0.01060286,0.5,1256,chr16,88003483,88003484,16,88003484


In [31]:
hla_related_genes = set(en_df[en_df['chromosome'] == 25]['gene'])
full_predicted_genes = set(predictions_df.index) - hla_related_genes
full_predicted_df = predictions_df.ix[full_predicted_genes]
full_predicted_df.columns = full_predicted_df.columns.map(lambda x: "-".join(x.split('-')[:2]))

In [33]:
full_predicted_df.to_csv(os.path.join(analysis_dir, "gtex.predixcan.elasticnet.minus-hla.tsv",
                        sep="\t")